In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping   



In [8]:
image_size= 224
Batch_size=32
epoch = 3
Data_Path= "/Users/ayushagrawal/Desktop/crop disease detection +frazile recomandton /pythonfile/train"


In [9]:
datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2  # Use 20% of data for validation
)

In [10]:
print("Loading Training Data...")
train_data = datagen.flow_from_directory(
    Data_Path, 
    target_size=(image_size, image_size),
    batch_size=Batch_size,
    class_mode='categorical',
    color_mode='rgb',      # <--- STRICTLY RGB
    subset='training',
    shuffle=True
)
print("TRAIN IMAGE SHAPE:", train_data.image_shape) 

Loading Training Data...
Found 11557 images belonging to 14 classes.
TRAIN IMAGE SHAPE: (224, 224, 3)


In [11]:
print("Loading Validation Data...")
val_data = datagen.flow_from_directory(
    Data_Path,
    target_size=(image_size, image_size),
    batch_size=Batch_size,
    class_mode='categorical',
    color_mode='rgb',      # <--- STRICTLY RGB
    subset='validation'
)


Loading Validation Data...
Found 2880 images belonging to 14 classes.


In [12]:
base_model = tf.keras.applications.EfficientNetB0(
    weights=None,
    include_top=False,
    input_shape=(image_size, image_size, 3) # <--- STRICTLY 3 CHANNELS
)

In [13]:
base_model.trainable = False


In [14]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(len(train_data.class_indices), activation='softmax') # Auto-detects number of classes
])

In [15]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [16]:
checkpoint = ModelCheckpoint(
    'BEST_CROP_MODEL.h5', 
    monitor='val_accuracy', 
    save_best_only=True, 
    mode='max', 
    verbose=1
)




In [17]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=3, 
    restore_best_weights=True
)

In [18]:
print("Starting Training...")
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epoch,
    callbacks=[checkpoint, early_stopping]
)

Starting Training...
Epoch 1/3
362/362 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step - accuracy: 0.1690 - loss: 2.6119
Epoch 1: val_accuracy improved from None to 0.17396, saving model to BEST_CROP_MODEL.h5


362/362 ━━━━━━━━━━━━━━━━━━━━ 155s 418ms/step - accuracy: 0.1720 - loss: 2.5867 - val_accuracy: 0.1740 - val_loss: 2.5430
Epoch 2/3
362/362 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step - accuracy: 0.1763 - loss: 2.5301
Epoch 2: val_accuracy did not improve from 0.17396
362/362 ━━━━━━━━━━━━━━━━━━━━ 152s 419ms/step - accuracy: 0.1737 - loss: 2.5160 - val_accuracy: 0.1740 - val_loss: 2.4934
Epoch 3/3
362/362 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step - accuracy: 0.1766 - loss: 2.4812
Epoch 3: val_accuracy did not improve from 0.17396
362/362 ━━━━━━━━━━━━━━━━━━━━ 155s 427ms/step - accuracy: 0.1737 - loss: 2.4807 - val_accuracy: 0.1740 - val_loss: 2.4692


In [19]:
print("Training Complete. Model saved as BEST_CROP_MODEL.h5")
model.save("CROP_MODEL.h5")

Training Complete. Model saved as BEST_CROP_MODEL.h5
